In [1]:
import pandas as pd 
import numpy as np 

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score



In [23]:
data = pd.read_csv("./houseprice.csv")

In [33]:
data.shape

(1460, 81)

In [36]:
numeric = ["int16", "int32", "int64", "float16","float32","float64"]
numerical_val = data.select_dtypes(numeric).columns
data = data[numerical_val]

In [37]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=["Id","SalePrice"],axis=1),data["SalePrice"], random_state = 0, test_size=0.3)
X_train.shape, X_test.shape
#drop xóa với axis = 0(row-index) ,axis= 1(column -labels) 

((1022, 36), (438, 36))

In [41]:
X_train.reset_index(drop=True, inplace=True)
X_train.fillna(0, inplace=True)

In [42]:
X_test.reset_index(drop=True, inplace =True)
X_test.fillna(0, inplace=True)

In [43]:
rf =RandomForestRegressor(n_estimators=100, max_depth=3, random_state=2, n_jobs=12)
# n_estimator : số cây
# max_depth: độ sâu của mỗi cây
# n_jobs: thực hiện đồng thời trên các luồng
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=3, n_jobs=12, random_state=2)

In [44]:
mean_squared_error(y_train, rf.predict(X_train),squared=False)

33977.057042182314

In [51]:
performance_shift = []
root_mse = mean_squared_error(y_train,rf.predict(X_train),squared=False)
for feature in X_train.columns:
    X_train_c =  X_train.copy()
    
    X_train_c[feature] = X_train_c[feature].sample(frac=1, random_state=11).reset_index(drop=True)
    
    shuff_rsme = mean_squared_error(y_train, rf.predict(X_train_c),squared=False)
    diff = root_mse - shuff_rsme
    performance_shift.append(diff)
    

In [52]:
feature_important = pd.Series(performance_shift)
feature_important.index = X_train.columns
feature_important.head()

MSSubClass         0.000000
LotFrontage       18.431053
LotArea         -253.112734
OverallQual   -45485.190691
OverallCond        0.000000
dtype: float64

In [54]:
feat = feature_important[feature_important < 0].index

In [55]:
X_train = X_train[feat]
X_test = X_test[feat]

In [56]:
rf =RandomForestRegressor(n_estimators=100, max_depth=3, random_state=2, n_jobs=12)
# n_estimator : số cây
# max_depth: độ sâu của mỗi cây
# n_jobs: thực hiện đồng thời trên các luồng
rf.fit(X_train, y_train)
mean_squared_error(y_train, rf.predict(X_train),squared=False)


33969.403723258234

In [57]:
mean_squared_error(y_test, rf.predict(X_test),squared=False)

39156.413985496656